# Gita Adviser

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://c4.wallpaperflare.com/wallpaper/661/37/264/god-lord-krishna-wallpaper-preview.jpg"> 
</p>
</div>

## Description:

The **Gita Advicer** app is designed to provide spiritual solutions to life’s challenges based on the teachings of the Bhagavad Gita. Users can input their life situations, and the app generates thoughtful responses rooted in Gita’s philosophy.

It focuses on:


- `Input Handling`: Accepts user-submitted scenarios or concerns.

- `Spiritual Insights`: Maps inputs to relevant verses and teachings from the Bhagavad Gita.

- `Guidance Output`: Provides users with reflective advice to inspire clarity and peace of mind.

This app offers practical wisdom and promotes mindfulness for everyday situations.


## Plan

- We have a CSV file of the Bhagvad Gita Shlokas and their meanings and other necessary data, we first load the data into a Dataframe object.

- Have a function which takes user input or life situation, searches the dataset, and return relevant shlokas.

- LLM to format, structure, and add reasoning to present the final output to the user.

- Render the output as markdown format for easy consumption.



## Step 1: Environment Setup and Installation

This step installs dependencies from `requirements.txt` and verifies the `OPENAI_API_KEY` environment variable.  

If installation fails, it retries up to 3 times before exiting.  

Once complete, it clears the output and confirms setup success.  


In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed,retries
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True, dotenv_path=".env")

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

## Step 2: Dataset Handling

#### **Loading a Dataset**

- The `load_dataset` function loads a CSV file from the provided path into a `pandas DataFrame`.  

- It handles two exceptions:  

  - **FileNotFoundError:** Displays a "File not found" message and prints the stack trace.  

  - **Generic Exception:** Catches all other errors, prints an error message, and shows the stack trace.

#### **Saving a Dataset**

- The `save_dataset` function saves a `pandas DataFrame` to the specified path.  

- It prints a success message upon saving or an error message with a stack trace if saving fails.  

#### **Loading Bhagavad Gita Data**

- The `load_bhagavad_gita` function loads Bhagavad Gita data from the hardcoded path `"data/bhagavad_gita/shlokas_with_context_v2.csv"` using the `load_dataset` function.

This code provides a structured way to manage datasets with proper error handling for better debugging and reliability.


In [ ]:
import pandas as pd
import traceback

GITA_DATASET_PATH = "data/bhagavad_gita/shlokas.csv"


def load_dataset(path: str) -> pd.DataFrame:
    """
    Load the dataset from the given path and return it as a pandas DataFrame.

    Args:
        path (str): The path to the dataset.

    Returns:
        pd.DataFrame: The dataset as a pandas DataFrame
    """
    try:
        return pd.read_csv(path)
    except FileNotFoundError:
        print("File not found. Please check the path.")
        traceback.print_exc()
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return None


def save_dataset(dataset: pd.DataFrame, path: str) -> None:
    """
    Save the dataset to the given path.

    Args:
        dataset (pd.DataFrame): The dataset to save.
        path (str): The path to save the dataset to.
    """
    try:
        dataset.to_csv(path)
        print("Dataset saved successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        traceback.print_exc()


def load_bhagavad_gita():
    dataset_path = "data/bhagavad_gita/shlokas_with_context_v2.csv"
    return load_dataset(path=dataset_path)

## Step 3: Loading and Previewing the Dataset

#### **Loading the Bhagavad Gita Data**

- The `load_bhagavad_gita` function is called, and its result is stored in the variable `df`.  

- This function loads the Bhagavad Gita dataset from the file `"data/bhagavad_gita/shlokas_with_context_v2.csv"`.  

#### **Previewing the Data**

- `df.head()` is used to display the first 5 rows of the loaded dataset.  

- This provides a quick look at the data to ensure it is loaded correctly and understand its structure.  


In [ ]:
df = load_bhagavad_gita()

df.head()

## Step 4: Adding Real-Life Context to the Bhagavad Gita Data  

#### **1. Function: `create_context`**  

- **Purpose:** Creates a real-life context for each row of the Bhagavad Gita dataset.  

- **Steps:**  
  - Extracts shloka details (chapter, verse, meanings, word meanings) from the row.  

  - Constructs a system prompt instructing the AI to relate the shloka to real-life situations.  

  - Sends the prompt to the `openai` model `gpt-4o-mini`.  

  - Returns the generated context.  

- **Error Handling:** Prints errors and returns "NA" if any issues occur during the process.  

#### **2. Function: `add_life_context_to_gita`**  

- **Purpose:** Adds real-life context to all shlokas in the Bhagavad Gita dataset.  

- **Steps:**  
  - Loads the dataset using `load_bhagavad_gita`.  

  - Applies the `create_context` function to each row to generate a "context" column.  

  - Saves the augmented dataset to `"data/bhagavad_gita/shlokas_with_context.csv"`.  

  - Returns the updated DataFrame.  


In [ ]:
import pandas as pd
from openai import OpenAI
import traceback

openai = OpenAI()

DEFAULT_OPENAI_MODEL = "gpt-4o-mini"


def create_context(row: pd.Series) -> str:
    """
    Create a real-life context for the given row.

    Args:
        row (pd.Series): The row containing the Gita shloka.

    Returns:
        str: The real-life context for the given row.
    """
    try:
        cur_idx = row["ID"]
        print(f"{cur_idx}: Creating context for: {row['Shloka']}")
        shloka_text = f"""
            Chapter: {row['Chapter']}
            Verse: {row['Verse']}
            Shloka: {row['Shloka']}
            Hindi Meaning: {row['HinMeaning']}
            English Meaning: {row['EngMeaning']}
            Word Meanings: {row['WordMeaning']}
        """

        system_prompt = f"""
            You are Bhagavad Gita Maverick and you are explaining the following shloka to a friend who is going through a tough time.

            Given a Bhagavad Gita shloka, provide a real-life context for the shloka.
            This should include situations where the teaching is applicable so that the person can relate to the learnings. 
            Provide your answer in General Language. 
        """

        user_prompt = f"""
            Shloka: {shloka_text}
        """

        response = openai.chat.completions.create(
            model=DEFAULT_OPENAI_MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.84,
        )

        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred when adding real-life context: {e}")
        traceback.print_exc()
        return "NA"


def add_life_context_to_gita() -> pd.DataFrame:
    """
    Augments the Bhagavad Gita dataset with real-life context to Gita shlokas.

    Args:
        df (pd.DataFrame): The dataset containing the Gita shlokas.

    Returns:
        pd.DataFrame: The augmented dataset.
    """
    updated_path = "data/bhagavad_gita/shlokas_with_context.csv"
    gita = load_bhagavad_gita()
    total_rows = gita.shape[0]
    print(f"Total rows: {total_rows}")
    gita["context"] = gita.apply(create_context, axis=1)
    save_dataset(gita, updated_path)
    return gita

## Step 5: Calling `add_life_context_to_gita`

- Loads the Bhagavad Gita dataset.  

- Generates real-life contexts for each shloka.  

- Adds a "context" column.  

- Saves the updated dataset and returns it.  


In [ ]:
# add_life_context_to_gita()

## Step 6: Loading the Enhanced Dataset  

- Loads the dataset with added shloka contexts.  

- Displays the first few rows for verification.  


In [ ]:
## This dataset is generated by ChatGPT which was much faster: our code would take 1hr vs ~1 min for ChatGPT

dataset_with_context = load_dataset("data/bhagavad_gita/shlokas_with_context_v2.csv")

dataset_with_context.head()

## Step 7: Bhagavad Gita Data Management Using ChromaDB

This code integrates the Bhagavad Gita dataset into ChromaDB for efficient storage and retrieval.

### `load_bhagavad_gita_into_db()`

- Loads the Bhagavad Gita dataset into ChromaDB by iterating over each row.  

- Converts each row into a document containing chapter, verse, translations, word meanings, and context.  

- The documents are indexed using their `ID` and added to the `"bhagavad_gita"` collection.  

- Returns the total number of documents loaded or 0 if an error occurs.  

### `query_shloka(query: str, n=5)`

- Executes a query on the ChromaDB collection for shlokas matching the given text.  

- Returns up to `n` matching documents.  

- If an error occurs, prints an error message and returns an empty list.  


In [ ]:
import chromadb
import traceback

chroma_client = chromadb.PersistentClient()

collection = chroma_client.get_or_create_collection(name="bhagavad_gita")


def load_bhagavad_gita_into_db():
    """
    Load the Bhagavad Gita dataset into the Chroma database.

    Args:
        None

    Returns:
        int: The number of documents loaded into the database.
    """
    try:
        df = load_bhagavad_gita()
        documents = []
        ids = []

        for _, row in df.iterrows():
            shloka_text = f"""
                ID: {row["ID"]}
                Chapter: {row['Chapter']}
                Verse: {row['Verse']}
                Shloka: {row['Shloka']}
                Hindi Meaning: {row['HinMeaning']}
                English Meaning: {row['EngMeaning']}
                Word Meanings: {row['WordMeaning']}
                Context: {row['context']}
            """

            doc_id = row["ID"]
            documents.append(shloka_text)
            ids.append(doc_id)

        collection.upsert(documents=documents, ids=ids)
        return len(documents)
    except Exception as e:
        print(f"An error occurred when loading the dataset into the database: {e}")
        traceback.print_exc()
        return 0


def query_shloka(query: str, n=5):
    """
    Query the Bhagavad Gita dataset for the given query.

    Args:
        query (str): The query to search for in the dataset.

    Returns:
        list: The list of documents matching the query.
    """
    try:
        results = collection.query(query_texts=[query], n_results=n)
        documents = results["documents"][0]
        return documents
    except Exception as e:
        print(f"An error occurred when querying the dataset: {e}")
        traceback.print_exc()
        return []

## Step 8: `load_bhagavad_gita_into_db()`

- This function loads the Bhagavad Gita dataset into the ChromaDB collection named `"bhagavad_gita"`.  

- It reads the dataset, converts each row to a structured document with details like `ID`, chapter, verse, translations, and context.  

- Each document is indexed using the corresponding `ID` and added to the database.  

- Returns the total number of documents loaded into the database or 0 if any error occurs.  


In [ ]:
load_bhagavad_gita_into_db()

## Step 9: Querying and Responding to Life Situations Using `ask_gita`

- The `ask_gita()` function takes a user query as input to search the Bhagavad Gita dataset for relevant shlokas.

- It uses `query_shloka(query, n=2)` to fetch two relevant shlokas.

- If no shlokas are found, it returns a message prompting the user to try a different query.

- The shlokas are formatted as text and included in a prompt to OpenAI, along with the user's query.

- A system prompt ensures the AI provides a kind, empathetic explanation of the teachings.

- The final response is generated using OpenAI's model and returned to the user.


In [ ]:
from openai import OpenAI

openai = OpenAI()

DEFAULT_SHLOKA_COUNT = 5


def ask_gita(query: str, shloka_count=DEFAULT_SHLOKA_COUNT) -> str:
    """
    Query the Bhagavad Gita dataset for the given query.

    Args:
        query (str): The query to search for in the dataset.
        shloka_count (int) : Number of shlokas to be generated.

    Returns:
        str: The response to the query.
    """
    shlokas = query_shloka(query, n=shloka_count)

    if not shlokas:
        return "I couldn't find any relevant shlokas for your query. Please try again with a different query."

    print(shlokas)

    sholkas_text = "\n".join(shlokas)

    system_prompt = f"""
                You are Bhagavad Gita Maverick and you are explaining the following shloka to a friend who is going through a tough time.
                Given a user query and some Bhagavad Gita shlokas, provide a response that helps the user understand the teachings of the Gita.
                The answer should be relevant and applicable to the user's situation.
                Be kind, respectful, and empathetic in your response.
            """

    user_prompt = f"""
                User Query: {query}
                Shlokas Text: {sholkas_text}
                """

    response = openai.chat.completions.create(
        model=DEFAULT_OPENAI_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.84,
    )

    return response.choices[0].message.content

## Step 10: Querying Shlokas for a Life Situation

- A life situation is represented using the `life_situation` variable to describe someone facing significant stress and unhealthy coping mechanisms.

- The `ask_gita()` function is called with this life situation to provide guidance based on Bhagavad Gita teachings.

- The response is stored in the `response` variable, formatted in Markdown, and displayed using `display(Markdown(response))`.

- This approach helps offer meaningful spiritual advice for navigating difficult life circumstances.


In [ ]:
from IPython.display import Markdown, display

shloka_count = 5

life_situation = f"""
My friend Jill Ba is going through money addiction. 
He thinks about money and wealth all day and doesn't do anything else. 
He neglects his wife and children and family duties.
"""

response = ask_gita(life_situation, shloka_count=shloka_count)

display(Markdown(response))

In [ ]:
life_situation = f"""
My friend Jill Ba is going through money addiction. 
He thinks about money and wealth all day and doesn't do anything else. 
He neglects his wife and children and family duties.
"""

shlokas = query_shloka(life_situation, n=5)

shlokas

In [ ]:
life_situation = f"""
Stacy, my 25-year-old friend, has been struggling a lot recently. 
He's under a lot of stress from work, relationships, and personal issues. 
To cope, he's been drinking heavily, smoking, and relying on other unhealthy distractions. 
It's like he's trapped in a cycle, trying to escape the pressure, but it's only making things worse. 
I can see that deep down, he knows it’s not helping, but he just doesn’t know how to get out of it.
"""

shlokas = query_shloka(life_situation, n=5)

shlokas

## Conclusion

The **Gita Adviser** app combines spiritual wisdom and practical advice from the Bhagavad Gita, tailored to real-life challenges. Using OpenAI and ChromaDB, it:

### Provides Personalized Guidance

- Delivers relevant shlokas based on user inputs.

### Enhances User Experience

- Makes teachings more relatable and actionable.

### Ensures Efficient Data Management

- Optimizes storage and retrieval for fast responses.

This app helps users navigate **life’s struggles**, **fostering peace** and **clarity**.


---

# Thank You for visiting The Hackers Playbook! 🌐

If you liked this research material;

- [Subscribe to our newsletter.](https://thehackersplaybook.substack.com)

- [Follow us on LinkedIn.](https://www.linkedin.com/company/the-hackers-playbook/)

- [Leave a star on our GitHub.](https://www.github.com/thehackersplaybook)

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="200px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>